In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00


#Dataset  
data를 하나씩 반환하며  
BertTokenzier를 통해 tokenziation후 input_ids, token_type_ids, attention mask를 tensor로 바꿔서 return  
또한 label이 spam일때는 1 ham일때는 0을 함께 return  

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer,BertTokenizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

data_path = "/content/drive/MyDrive/nlp-open-tutorial/6일차_배포/dataset/spam.csv"
data_df = pd.read_csv(data_path, encoding = "ISO-8859-1")

data_df = data_df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
train_df = data_df.loc[:400,:].reset_index()
test_df = data_df.loc[400:600,:].reset_index()

class myDataset(Dataset):
  def __init__(self, df, tokenizer) -> None:
      super().__init__()
      self.df = df
      self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

  def __len__(self):
      return len(self.df)

  def __getitem__(self, index):
      data = self.df.loc[index, "v2"]
      target = self.df.loc[index, "v1"]

      #data tokenization
      data = self.tokenizer(data, max_length=100,padding='max_length', truncation=True)

      #convert to tensor
      input_ids = torch.IntTensor(data['input_ids'])
      token_type_ids = torch.IntTensor(data['token_type_ids'])
      attention_mask = torch.IntTensor(data['attention_mask'])

      #labeling
      if target == "ham":
          label = 0
      elif target == "spam":
          label = 1

      return input_ids, token_type_ids, attention_mask, label


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

train_dataset = myDataset(train_df, tokenizer)
test_dataset = myDataset(test_df, tokenizer)

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

#잘 실행되는지 확인
for i in train_dataset:
  input_ids, token_type_ids, attention_mask, label= i
  print("input_ids: ",input_ids)
  print("token_type_ids: ",token_type_ids)
  print("attention_mask: ",attention_mask)
  break
print()

for batch in train_dataloader:
  input_ids, token_type_ids, attention_mask, label= batch
  print("batch_input_ids: ",input_ids.shape)
  print("batch_token_type_ids: ",token_type_ids.shape)
  print("batch_attention_mask: ",attention_mask.shape)
  print("batch_label",label.shape)
  break

input_ids:  tensor([  101,  3414,  1235,   179, 22497,  1403,  1553,   117,  4523,   119,
          119, 11651,  8009,  2165,  1178,  1107, 15430,  1548,   183,  1632,
         1362,  2495,   174,   171,  9435,  2105,   119,   119,   119,   140,
         2042,  1175,  1400,  1821,  4474, 20049,  1204,   119,   119,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       dtype=torch.int32)
token_type_ids:  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    

##Model
BertModel을 이용해서 구현  
BertModel의 cls token에 대한 output인 pooler_output에 linear를 통해 2차원 벡터로 변환  
BertModel: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel

In [13]:
from torch import nn
from transformers import BertModel

class myModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.linear = nn.Linear(768, 2)

    def forward(self, input_ids, token_type_ids, attention_mask):
        output = self.bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        output = self.linear(output.pooler_output)
        return output

#잘 실행되는지 확인
model = myModel()

for batch in train_dataloader:
  input_ids, token_type_ids, attention_mask, label= batch
  output = model(input_ids, token_type_ids, attention_mask)
  print(output)
  break

tensor([[-0.1423,  0.3380],
        [-0.1835,  0.1116],
        [-0.1691,  0.2127],
        [-0.1786,  0.2724],
        [-0.1205, -0.0519],
        [-0.1281, -0.0236],
        [-0.0960,  0.0621],
        [-0.1548,  0.1852],
        [-0.1380,  0.2998],
        [-0.1089,  0.0225],
        [-0.1402,  0.0565],
        [-0.1466,  0.2250],
        [-0.1642,  0.2569],
        [-0.1488,  0.0402],
        [-0.1393,  0.3535],
        [-0.1182,  0.0668]], grad_fn=<AddmmBackward0>)


#Train and test
cross entropy loss와 Adam을 통해 train

In [22]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import tqdm
from tqdm.auto import tqdm

model = myModel().cuda()

optimizer = Adam(model.parameters(), lr=0.001)
lf = CrossEntropyLoss()

for e in range(5):
  print("\nepoch ", e)
  epoch_loss = 0
  train_correct = 0

  model.train()

  for batch in tqdm(train_dataloader):
    optimizer.zero_grad()

    #data GPU에 올려주기
    input_ids, token_type_ids, attention_mask, target= batch
    input_ids=input_ids.cuda()
    token_type_ids=token_type_ids.cuda()
    attention_mask=attention_mask.cuda()
    target=target.cuda()

    #결과 도출 및 정답 수 계산
    output = model(input_ids, token_type_ids, attention_mask)
    pred_label = torch.argmax(output, dim=-1)
    train_correct += sum(pred_label == target)

    loss = lf(output, target)

    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()

  # print(train_correct)
  print("train loss", epoch_loss/len(train_dataloader))
  print("train acc", train_correct/len(train_dataset))

  test_loss = 0
  test_correct = 0

  model.eval()

  with torch.no_grad():
    for bacth in tqdm(test_dataloader):
      input_ids, token_type_ids, attention_mask, label= batch
      input_ids=input_ids.cuda()
      token_type_ids=token_type_ids.cuda()
      attention_mask=attention_mask.cuda()
      target=target.cuda()

      output = model(input_ids, token_type_ids, attention_mask)
      pred_label = torch.argmax(output, dim=-1)
      test_correct += sum(pred_label == target)

      loss = lf(output, target)
      test_loss += loss.item()

  print("test loss", test_loss/len(test_dataloader))
  print("test acc", test_correct/len(test_dataset))


epoch  0


  0%|          | 0/26 [00:00<?, ?it/s]

train loss 0.6158625603868411
train acc tensor(0.8229, device='cuda:0')


  0%|          | 0/13 [00:00<?, ?it/s]

test loss 0.4600140154361725
test acc tensor(0.0647, device='cuda:0')

epoch  1


...

test loss 1.226283073425293
test acc tensor(0., device='cuda:0')

epoch  4


  0%|          | 0/26 [00:00<?, ?it/s]

train loss 0.5339483789239938
train acc tensor(0.7930, device='cuda:0')


  0%|          | 0/13 [00:00<?, ?it/s]

test loss 1.2557886838912964
test acc tensor(0., device='cuda:0')
